In [ ]:
from huggingface_hub import login
login()  # This will ask for your token


In [ ]:
!pip install -U bitsandbytes transformers accelerate


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load model in 4-bit mode (reduces VRAM usage)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


In [ ]:
!pip install pymupdf faiss-cpu sentence-transformers transformers accelerate bitsandbytes


In [ ]:
import fitz  # PyMuPDF for PDF reading
import faiss  # For fast text similarity search
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

#  Load Llama-2 **without 4-bit quantization**
model_name = "meta-llama/Llama-2-7b-chat-hf"

#  Force CPU execution
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"": "cpu"}  # Forces CPU usage
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

#  Create text-generation pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load SentenceTransformer for text embedding
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

#  Load and extract text from PDF
pdf_path = "/content/10-Q4-2024-As-Filed.pdf"
doc = fitz.open(pdf_path)

text_chunks = [page.get_text("text") for page in doc]
doc_text = " ".join(text_chunks)

#  Increase chunk size from 500 → 1000 for better context
chunk_size = 1000
chunks = [doc_text[i:i+chunk_size] for i in range(0, len(doc_text), chunk_size)]

#  Convert chunks to embeddings
chunk_embeddings = np.array([embed_model.encode(chunk) for chunk in chunks])

#  Check embeddings shape
if chunk_embeddings.shape[0] == 0:
    print("Error: No embeddings generated!")
else:
    print(f"Embeddings Shape: {chunk_embeddings.shape}")

#  Create FAISS index
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings)

#  Store chunk mapping
chunk_mapping = {i: chunk for i, chunk in enumerate(chunks)}

#  Function to retrieve answer from FAISS
def retrieve_from_faiss(query):
    query_embedding = embed_model.encode(query).reshape(1, -1)
    D, I = index.search(query_embedding, k=10)  #  Retrieve top 10 results

    print("\nFAISS Debugging:")
    print(f"Query: {query}")
    print(f"Similarity Scores: {D[0]}")

    retrieved_texts = [chunk_mapping[i] for i in I[0] if i != -1]

    #  Set a relevance threshold (Lower is better)
    similarity_threshold = 1.0  # If all results are above 1.0, assume FAISS failed

    best_score = min(D[0])  # Take the best (lowest) similarity score

    if best_score > similarity_threshold:
        print("FAISS did not find a relevant document.")
        return None  # Now Llama will handle general questions

    return " ".join(retrieved_texts[:3])  # Return top 3 chunks for better context

# Function to get Llama response
def ask_llama(prompt):
    response = llm_pipeline(prompt, max_length=200, num_return_sequences=1)
    return response[0]['generated_text']

#  Main function to handle user query
def query_rag_system(user_question):
    retrieved_doc = retrieve_from_faiss(user_question)

    if retrieved_doc:
        print(f"\nUser Question: {user_question}")
        print(f"Answer from document: {retrieved_doc}")
    else:
        print(f"\nUser Question: {user_question}")
        print(f"Answer from Llama: {ask_llama(user_question)}")  # ✅ Now queries Llama-2 instead of showing "No relevant answer"


In [ ]:
#  Example questions related to Apple’s 10-K report
example_questions = [
    "What sustainability initiatives has Apple implemented?",
    "What were Apple’s total assets and liabilities at the end of 2024?",
    "Has Apple reported any legal issues or ongoing litigation in 2024?"
]

#  Run example queries
for q in example_questions:
    query_rag_system(q)



FAISS Debugging:
Query: What sustainability initiatives has Apple implemented?
Similarity Scores: [0.88739234 0.98691994 1.0103631  1.0284477  1.0311035  1.0636687
 1.068084   1.0724581  1.0777228  1.0818608 ]

User Question: What sustainability initiatives has Apple implemented?
Answer from document: ’s success. The Company recognizes its people are most likely to thrive when they have the resources to meet their 
needs and the time and support to succeed in their professional and personal lives. In support of this, the Company offers a wide 
variety of benefits for employees around the world, including health, wellness and time away.
Growth and Development
The Company invests in resources to help its people develop and achieve their career goals. The Company offers programs 
through Apple University on leadership, management and influence, as well as Apple culture and values. Team members can 
also take advantage of online classes for business, technical and personal development, as

In [ ]:
# ✅ Example questions
example_questions = [

    "who is the current Prime minister of India?",
]

# ✅ Run example queries
for q in example_questions:
    query_rag_system(q)



FAISS Debugging:
Query: who is the current Prime minister of India?
Similarity Scores: [1.6910474 1.7179368 1.7256262 1.7310582 1.7373565 1.7462183 1.7543325
 1.7558703 1.7616369 1.7675813]
FAISS did not find a relevant document.

User Question: who is the current Prime minister of India?
Answer from Llama: who is the current Prime minister of India?
 Unterscheidung zwischen "Who" und "Whom"  Who is the current Prime Minister of India? The current Prime Minister of India is Narendra Modi. He has been in office since May 2014 and was re-elected in the 2019 general elections. Prior to becoming Prime Minister, Modi served as the Chief Minister of Gujarat from 2001 to 2014.


In [ ]:
# ✅ Example questions
example_questions = [

    "who is the current Prime minister of India?",
]

# ✅ Run example queries
for q in example_questions:
    query_rag_system(q)



FAISS Debugging:
Query: who is the current Prime minister of India?
Similarity Scores: [1.6910474 1.7179368 1.7256262 1.7310582 1.7373565 1.7462183 1.7543325
 1.7558703 1.7616369 1.7675813]
FAISS did not find a relevant document.

User Question: who is the current Prime minister of India?
Answer from Llama: who is the current Prime minister of India?
 Einzeln, the current Prime Minister of India is Narendra Modi. He was sworn in as the Prime Minister of India for the second time on May 30, 2019, after leading the Bharatiya Janata Party (BJP) to a landslide victory in the 2019 Indian general elections. Modi has been in office since May 2014 and has been instrumental in shaping India's economic and political landscape during his tenure.


In [ ]:
example_questions = [
    "What is mitochondria?"
]

# ✅ Run example queries
for q in example_questions:
    print("\nUser Question:", q)
    query_rag_system(q)


User Question: What is mitochondria?

FAISS Debugging:
Query: What is mitochondria?
Similarity Scores: [1.681306  1.7918278 1.7975855 1.816612  1.818956  1.8239379 1.8263508
 1.8264081 1.830353  1.8324184]
FAISS did not find a relevant document.

User Question: What is mitochondria?
Answer from Llama: What is mitochondria?
 Bedeutung von Mitochondria

Mitochondria are organelles found in the cells of most eukaryotes (including animals, plants, and fungi). They are often referred to as the "powerhouses" of cells because they generate most of the energy that cells need to function. Mitochondria are responsible for producing energy in the form of ATP (adenosine triphosphate), which is the primary energy currency of cells.

Mitochondria have a unique structure, with an outer and an inner membrane, and a matrix (the space between the two membranes) that contains the enzymes and other molecules necessary for energy production. They also have their own DNA, known as mtDNA, which is separate 